In [56]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
import os


In [75]:
# Download and load the data
#import keras
import os

f_path_1 = "data/Euro_Salary.csv"
#f_path_1 = "Euro_Salary.csv"
url_1 = "https://github.com/AkeemSemper/ml_data/raw/main/Euro_Salary.csv"
if not os.path.exists(f_path_1):
    file_1 = keras.utils.get_file(f_path_1, url_1)
df = pd.read_csv(f_path_1)

NameError: name 'keras' is not defined

# Assignment 2 - Regression
### Predict the TOTAL COMPENSATION for this year. 

The data file provided is a salary survey for tech workers in Europe. We want to predict the total amount of compensation they bring in each year, based off of the details of their work. 

Some notes that will be important:
<ul>
    <li>The total compensation will need to be constructed, there is a column for salary, "Yearly brutto salary (without bonus and stocks) in EUR", as well as a column for bonus compensation, "Yearly bonus + stocks in EUR". 
    <li>Some categorical variables will need some work, and there isn't generally an exact answer. The main concern is things with categories that have a bunch of values with a very small count. For example, if there is only 1 person in City X, then that value likely needs to be addressed. We don't want it encoded into a new column of one 1 and thousands of 0s. 
    <li>There is an article exploring some of the data here: https://www.asdcode.de/2021/01/it-salary-survey-december-2020.html
    <li>Imputation and a bit of data manipulation will be required. 
    <li>Use any regression method you'd like. Some ones are closely related to what we've done, you may want to look at them, e.g. ExtraTreesRegressor. 
    <li>Initial accurracy, and potentially final accuracy, may not be great. When I made a plain model will little optimization the errors were large and the R2 was low. There is lots of room for optimization. 
    <li>Research challenge - try some work on the target, look into TransformedTargetRegressor and see if that helps. Recall in stats when we had skewed distributions... Maybe it helps, maybe it doesn't. 
    <li>EDA and data prep are up to you - you'll probably need to do a little exploring to figure out what cleanup is needed. When I did it, I did things kind of iteratively when I did it. For example, look at the value counts, figure out how to treat the different categories, clean something up, look at the results, potentially repeat if needed. After you figure out what needs to be done, you may be able to take some of those steps and incorporate them into a pipeline to be cleaner....
    <li><b>CRITICAL - Please make sure you publish it after having run it, all the output should be showing.</b>
</ul>

### Details and Deliverables

You'll need to build code to produce the predictions. In particular, there's a few things that'll be marked:
<ul>
    <li>Please add a "presentation version" at the bottom, where you show what you did, and the results. Basically, you start with the original data, you do some work to figure out what's needed, you try a few models and select the best. At the bottom, put what <i>you actually settled on</i>, i.e. after all the figuring and exploring, here's the code that goes from raw data to final results, and here's what the results were. I should be able to read this part and understand what you did clearly:
    <ul>
        <li> Please make a pipeline that does the prep work - you may need some exploration or several trials before settling on what exactly to use, that's normal. Once you've settled, build that into a pipeline so it's clear and repeatable.
        <li> What you settled on for data cleaning, along with what prompted it. 
        <li> Feature Selection - Please identify what you did for feature selection. No need for a long explaination, something along the lines of "I did X, and the result was that 4 features were removed". Try at least 2 things. 
        <li> Model selection - between selecting a model style and tuning it with hyperparameters, what did you test and what won?
        <li> Overall, how good was your model and what things may make sense to try to do even better? 
        <li> If you could use titles/bullet points I'd really appreciate it. 
    </ul>
    <li>Grade Breakdown:
    <ul>
        <li> Code is readable, there are comments: 20%
        <li> Explaination as defined above: 60% (20% each point)
        <li> Accuracy: 20% As compared to everyone else. This will be generously graded, I won't be surprised if overall accuracy is low for most people. 
    </ul>
</ul>

<b>The biggest challenge here is translating the data into something useful and clean. This will probably require a bit of exploration, examining the data, thinking about what it means, trying something, then making a model to see what the results are. In particular, think about what value some of the less clean bits of data my hold - binning/grouping, numerical transformations, outlier removal, etc... are all likely to be useful somewhere. You almost certainly need to look at it column by column and make a decision, I'll apologize up front, it isn't the most fun process in the world. There is not one specific correct answer.</b>

In [58]:
#Load Data
df = pd.read_csv("data/Euro_Salary.csv")
#df = pd.read_csv("Euro_Salary.csv")
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Timestamp,1253,1248,24/11/2020 13:55:19,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,1226.0,NaN,NaN,NaN,32.509788,5.663804,20.0,29.0,32.0,35.0,69.0
Gender,1243,3,Male,1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,1253,119,Berlin,681,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Position,1247,148,Software Engineer,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total years of experience,1237,48,10,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Years of experience in Germany,1221,53,2,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Seniority level,1241,24,Senior,565,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Your main technology / programming language,1126,256,Java,184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other technologies/programming languages you use often,1096,562,Javascript / Typescript,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
df[393:396]

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
393,24/11/2020 19:15:02,30.0,Male,Moscow,Software Engineer,5,0,Middle,C,C/C++,14712.0,0,30,Full-time employee,Unlimited contract,Russian,101-1000,Product
394,24/11/2020 19:15:49,33.0,Male,Berlin,Product Manager,5,5,Senior,NaN,Python,70000.0,800,30,Full-time employee,NaN,German,101-1000,Product
395,24/11/2020 19:19:30,35.0,Male,Berlin,QA Engineer,11,10,Senior,Java,"Python, Javascript / Typescript, .NET, Java / ...",74400.0,NaN,30,Full-time employee,Unlimited contract,English,101-1000,Product


In [60]:
#df.info()
df["Other technologies/programming languages you use often"].value_counts()

Other technologies/programming languages you use often
Javascript / Typescript                                                       44
Python                                                                        37
SQL                                                                           31
AWS, Docker                                                                   16
Kotlin                                                                        15
                                                                              ..
PHP, SQL, Kubernetes, Docker                                                   1
Python, Go, AWS, Docker                                                        1
Python, Javascript / Typescript, Java / Scala, Go, AWS, Kubernetes, Docker     1
Python, Javascript / Typescript, .NET, Java / Scala, Kubernetes, Docker        1
Javascript / Typescript, .NET, Kubernetes, Docker                              1
Name: count, Length: 562, dtype: int64

In [61]:
df.sample(20)

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
1102,06/12/2020 21:17:41,32.0,Female,Berlin,Computational linguist,5,5,Middle,Java,"Python, Docker",50400.0,0,29,Full-time employee,Unlimited contract,English,11-50,Product
1107,07/12/2020 08:28:00,27.0,Female,Berlin,Data Scientist,5,2,Senior,Python,"SQL, AWS, Google Cloud, Docker",75000.0,0,24,Full-time employee,NaN,English,101-1000,Product
387,24/11/2020 18:59:38,24.0,Male,Munich,Software Engineer,3,2,Middle,Java/C++,"Python, C/C++, Javascript / Typescript, Java /...",77600.0,43500,30,Full-time employee,Unlimited contract,English,1000+,Product
366,24/11/2020 18:32:14,24.0,Male,Aachen,HiWi,3,2,Junior,Python,"Python, C/C++",10164.0,NaN,14,Part-time employee,Temporary contract,English,11-50,University
608,25/11/2020 14:27:14,29.0,Male,Berlin,Backend Developer,5,1,Senior,Java,"Java / Scala, SQL, AWS, Kubernetes, Docker",70000.0,0,27,Full-time employee,Unlimited contract,English,1000+,Product
191,24/11/2020 12:48:17,30.0,Female,Berlin,QA Engineer,5,3,Middle,Python,"Python, Kubernetes, Docker, Ansible",50000.0,4000,30,Full-time employee,Unlimited contract,German,101-1000,Product
251,24/11/2020 14:17:20,31.0,Male,Berlin,QA Engineer,10,5,Senior,nothing,Python,60000.0,77000,28,Full-time employee,Unlimited contract,English,1000+,Product
70,24/11/2020 11:40:05,29.0,Male,Konstanz area,Mobile Developer,6,3,Middle,android,Kotlin,53000.0,NaN,26,Full-time employee,Unlimited contract,English,11-50,Startup
613,25/11/2020 14:37:07,37.0,Male,Berlin,Embedded Software Engineer,15,1,Senior,C/C++,"Python, C/C++",78000.0,0,28,Full-time employee,Unlimited contract,English,1000+,Product
1175,18/12/2020 13:14:39,33.0,Male,Berlin,Software Engineer,7,7,Senior,C++,Javascript / Typescript,65900.0,15900,30,Full-time employee,Unlimited contract,English,1000+,Product


# Answers and Explainations
(Expand/modify as needed)

### Here's the Data Cleaning Steps I Used

### Here's my Tuning/Feature Selection Steps

### Here's my Model's Performance

### Here's my Final Conclusion on What Worked Best

In [62]:
import pandas as pd
import numpy as np

#df = pd.read_csv("Euro_Salary.csv")
df = pd.read_csv("data/Euro_Salary.csv")
print(df.shape)
print(df.columns.tolist())
df.head()


(1253, 18)
['Timestamp', 'Age', 'Gender', 'City', 'Position ', 'Total years of experience', 'Years of experience in Germany', 'Seniority level', 'Your main technology / programming language', 'Other technologies/programming languages you use often', 'Yearly brutto salary (without bonus and stocks) in EUR', 'Yearly bonus + stocks in EUR', 'Number of vacation days', 'Employment status', 'Сontract duration', 'Main language at work', 'Company size', 'Company type']


,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
0,24/11/2020 11:14:15,26.0,Male,Munich,Software Engineer,5,3,Senior,TypeScript,"Kotlin, Javascript / Typescript",80000.0,5000,30,Full-time employee,Unlimited contract,English,51-100,Product
1,24/11/2020 11:14:16,26.0,Male,Berlin,Backend Developer,7,4,Senior,Ruby,NaN,80000.0,NaN,28,Full-time employee,Unlimited contract,English,101-1000,Product
2,24/11/2020 11:14:21,29.0,Male,Berlin,Software Engineer,12,6,Lead,Javascript / Typescript,"Javascript / Typescript, Docker",120000.0,120000,30,Self-employed (freelancer),Temporary contract,English,101-1000,Product
3,24/11/2020 11:15:24,28.0,Male,Berlin,Frontend Developer,4,1,Junior,Javascript,NaN,54000.0,NaN,24,Full-time employee,Unlimited contract,English,51-100,Startup
4,24/11/2020 11:15:46,37.0,Male,Berlin,Backend Developer,17,6,Senior,C# .NET,".NET, SQL, AWS, Docker",62000.0,NaN,29,Full-time employee,Unlimited contract,English,101-1000,Product


In [63]:
# strip whitespace in column names
df.columns = df.columns.str.strip()

# rename the Cyrillic-C column to normal ASCII name
df = df.rename(columns={"Сontract duration": "Contract duration"})

print(df.columns.tolist())


['Timestamp', 'Age', 'Gender', 'City', 'Position', 'Total years of experience', 'Years of experience in Germany', 'Seniority level', 'Your main technology / programming language', 'Other technologies/programming languages you use often', 'Yearly brutto salary (without bonus and stocks) in EUR', 'Yearly bonus + stocks in EUR', 'Number of vacation days', 'Employment status', 'Contract duration', 'Main language at work', 'Company size', 'Company type']


In [64]:
salary_col = "Yearly brutto salary (without bonus and stocks) in EUR"
bonus_col  = "Yearly bonus + stocks in EUR"

df[salary_col] = pd.to_numeric(df[salary_col], errors="coerce")
df[bonus_col]  = pd.to_numeric(df[bonus_col], errors="coerce")

df["total_compensation"] = df[salary_col] + df[bonus_col].fillna(0)
df[["total_compensation", salary_col, bonus_col]].head()



,total_compensation,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR
0,85000.0,80000.0,5000.0
1,80000.0,80000.0,NaN
2,240000.0,120000.0,120000.0
3,54000.0,54000.0,NaN
4,62000.0,62000.0,NaN


In [65]:
y = df["total_compensation"]

low  = y.quantile(0.01)
high = y.quantile(0.99)

df = df[(y >= low) & (y <= high)].copy()
print(df.shape, "after clipping")


(1228, 19) after clipping


In [66]:
df = df.drop(columns=[
    "Timestamp",
    salary_col,
    bonus_col,
    "Other technologies/programming languages you use often"
])


In [67]:
num_cols = [
    "Age",
    "Total years of experience",
    "Years of experience in Germany",
    "Number of vacation days"
]

for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")


In [68]:
target = "total_compensation"

cat_cols = [
    "Gender",
    "City",
    "Position",
    "Seniority level",
    "Your main technology / programming language",
    "Employment status",
    "Contract duration",
    "Main language at work",
    "Company size",
    "Company type"
]

X = df[num_cols + cat_cols].copy()
y = df[target].copy()


In [69]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [70]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

numeric_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", min_frequency=10))
])

preprocessor = ColumnTransformer([
    ("num", numeric_pipe, num_cols),
    ("cat", categorical_pipe, cat_cols)
])


In [71]:
from sklearn.linear_model import Ridge

ridge_model = Pipeline([
    ("prep", preprocessor),
    ("model", Ridge(alpha=1.0))
])


In [72]:
from sklearn.ensemble import ExtraTreesRegressor

etr_model = Pipeline([
    ("prep", preprocessor),
    ("model", ExtraTreesRegressor(
        n_estimators=400,
        random_state=42,
        n_jobs=-1
    ))
])


In [74]:
from sklearn.metrics import r2_score, mean_squared_error

def evaluate(model, name="model"):
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    r2 = r2_score(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))

    print(f"{name}: R2={r2:.4f}, RMSE={rmse:.2f}")
    return r2, rmse

evaluate(ridge_model, "Ridge")
evaluate(etr_model, "ExtraTrees")


Ridge: R2=0.2003, RMSE=30606.79
ExtraTrees: R2=-0.1223, RMSE=36257.91


(-0.12226101218178087, np.float64(36257.9115999385))

Here’s the Data Cleaning Steps I Used

I created a new target variable called total_compensation defined as:

total_compensation = base salary + bonus/stocks

Missing bonus values were treated as 0, since many respondents reported no bonus and leaving them as NaN would incorrectly drop those rows.

I cleaned the column names by stripping whitespace and correcting the malformed “Contract duration” column to avoid indexing errors.

Numeric columns such as Age, Total years of experience, Years of experience in Germany, and Number of vacation days were converted using pd.to_numeric with errors="coerce" to safely handle inconsistent entries. Invalid values were converted to NaN and later handled in the pipeline using imputation.

The dataset contained extreme outliers (for example, multi-billion EUR salary values). To prevent these unrealistic values from dominating the regression loss, I removed extreme observations using 1st–99th percentile clipping on the total_compensation variable.

I removed the column “Other technologies/programming languages you use often” because it contains over 500 unique free-text combinations, which would create a very sparse and high-dimensional feature space after one-hot encoding.

All imputation and encoding steps were handled inside a preprocessing pipeline to avoid data leakage and ensure reproducibility.

---

Here’s my Tuning / Feature Selection Steps

I split the dataset into training and testing sets using an 80/20 split.

I built a preprocessing pipeline that included:

* Median imputation and standard scaling for numeric features
* Most frequent imputation and one-hot encoding for categorical features

To reduce overfitting from rare categories, I used OneHotEncoder with min_frequency=10 so that very infrequent categories were grouped together instead of creating many sparse dummy variables.

I tested two models:

1. Ridge Regression (linear baseline model)
2. ExtraTreesRegressor (nonlinear tree-based model)

No aggressive manual feature elimination was performed beyond removing the high-cardinality free-text column. Feature complexity was mainly controlled through category grouping in the encoder.

---

Here’s my Model’s Performance

Ridge Regression:
R2 = 0.2003
RMSE ≈ 30,607 EUR

ExtraTreesRegressor:
R2 = -0.1223
RMSE ≈ 36,258 EUR

Ridge regression explains approximately 20% of the variance in total compensation. While this is not very high, it indicates that the model captures some meaningful structure in the data.

The ExtraTrees model produced a negative R2 score, which means it performed worse than simply predicting the mean salary for all observations. This suggests overfitting or instability given the sparse categorical feature space.

Overall, Ridge generalized better than the tree-based model on unseen data.

---

Here’s my Final Conclusion on What Worked Best

Ridge regression performed best among the tested models.

Although tree-based models can capture nonlinear relationships, in this dataset the feature space is high-dimensional and sparse due to categorical encoding. Additionally, salary depends on many latent factors that are not captured in the survey data (e.g., company brand, negotiation ability, stock equity structure, exact cost-of-living differences, performance bonuses).

Because of this noise and missing information, a simpler linear model generalized better than a nonlinear tree model.

The overall R2 value of approximately 0.20 suggests that predicting compensation from survey data is inherently challenging. Improvements could potentially be made by log-transforming the target variable, grouping cities by country or region, or engineering more refined seniority-related features.
